# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` を付けない）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**、表示は仕様どおりの列名と順序

## 1) 問題

* `Orders` から **最も多く注文を行った顧客の `customer_number`** を返す
  （テストでは最大が一意。Follow up: 同数最大が複数いても全件返す）

* 入力テーブル例:

  ```markdown
  Table: Orders
  +-----------------+----------+
  | Column Name     | Type     |
  +-----------------+----------+
  | order_number    | int      | -- PK
  | customer_number | int      |
  +-----------------+----------+
  ```

* 出力仕様:

  ```markdown
  +-----------------+
  | customer_number |
  +-----------------+
  ```

## 2) 最適解（単一クエリ）

> **ウィンドウ関数＋事前集計**で 1 クエリ。`OVER` 内の `ORDER BY` は順位付けのためで、最終 `SELECT` に `ORDER BY` は不要。

```sql
WITH cnt AS (
  SELECT
    customer_number,
    COUNT(*) AS order_cnt
  FROM Orders
  GROUP BY customer_number
),
win AS (
  SELECT
    customer_number,
    DENSE_RANK() OVER (ORDER BY order_cnt DESC) AS rnk
  FROM cnt
)
SELECT
  customer_number
FROM win
WHERE rnk = 1;

Runtime 429 ms
Beats 75.41%

```

* これで **一意最大**も**同数最大が複数**も対応（Follow up 充足）

## 3) 代替解

> **最大値をサブクエリで求めて一致フィルタ**。ウィンドウが重い環境や互換用に。

```sql
WITH cnt AS (
  SELECT customer_number, COUNT(*) AS order_cnt
  FROM Orders
  GROUP BY customer_number
),
mx AS (
  SELECT MAX(order_cnt) AS max_cnt FROM cnt
)
SELECT c.customer_number
FROM cnt AS c
JOIN mx ON c.order_cnt = mx.max_cnt;

Runtime 469 ms
Beats 41.83%

```

※ `NOT IN` は未使用。`ORDER BY ... LIMIT 1` でも実現できるが、本要件では**結果順は任意**かつ **最大同率全件**を自然に返せる上記方式が安全。

## 4) 要点解説

* **方針**: まず `customer_number` 単位で件数を集計 → 上位判定（`DENSE_RANK` か `MAX` 照合） → 仕様列のみ投影。
* **NULL / 重複**:

  * `customer_number` が NULL の行が存在するなら集計前に `WHERE customer_number IS NOT NULL` を入れる（問題文では想定外だが堅牢性の観点）。
  * `order_number` は PK のため重複はなし。
* **安定性**: 出力順は問わないため **最終 `ORDER BY` は不要**。上位選別はウィンドウ内で完結。

## 5) 計算量（概算）

* `GROUP BY` 集計: **O(N)**～**O(N log N)**（ヒープ/ソート次第）
* ウィンドウ `DENSE_RANK`（代替は `MAX` 照合）: 集計後のユニーク顧客数を M として **O(M log M)**（内部ソート含む）
  ※ 代替解（`MAX` 照合）は **O(M)** でやや軽量。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Orders] --> B[customer_number ごとに COUNT]
  B --> C[順位付け DENSE_RANK または MAX 照合]
  C --> D[rnk=1 または order_cnt=max_cnt を抽出]
  D --> E[出力 customer_number]
```


## 結論（用途別ベスト）

### 1) 最大が一意（本問題の前提）なら最短クエリが最速になりやすい

```sql
SELECT customer_number
FROM Orders
GROUP BY customer_number
ORDER BY COUNT(*) DESC
LIMIT 1;

Runtime 470 ms
Beats 41.18%

```

* 余計な CTE・結合・ウィンドウ不要。
* `GROUP BY` 後に件数降順で **先頭 1 件だけ**返すので、実装や実行計画的にもシンプル。

### 2) 同数最大をすべて返したい（Follow-up 汎用）

**ウィンドウ関数なし**で 1 クエリ：

```sql
SELECT customer_number
FROM Orders
GROUP BY customer_number
HAVING COUNT(*) = (
  SELECT COUNT(*) AS mx
  FROM Orders
  GROUP BY customer_number
  ORDER BY mx DESC
  LIMIT 1
);

Runtime 433 ms
Beats 72.43%

```

* 内側サブクエリで「最大件数」だけを 1 行取得 → 外側で一致フィルタ。
* あなたの `cnt→mx→JOIN` 版よりも結合が無いぶん軽くなることが多いです。

### 3) ウィンドウ関数派（可読性重視）なら CTE をやめて 1 段で

```sql
SELECT customer_number
FROM (
  SELECT
    customer_number,
    DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS rnk
  FROM Orders
  GROUP BY customer_number
) t
WHERE rnk = 1;

Runtime 461 ms
Beats 48.24%

```

* `COUNT(*)` を直接 `ORDER BY` に使い、そのまま `DENSE_RANK`。
* 中間 `cnt` CTE のマテリアライズを避けられる分だけ有利になる場合があります。

---

## 実務チューニングのヒント

1. **インデックス**

```sql
CREATE INDEX idx_orders_customer ON Orders(customer_number);
```

* `GROUP BY customer_number` の集約が大幅にラクになります（全表スキャン回避/ソート削減）。

2. **CTE は必要最小限に**
   MySQL 8 では CTE がマテリアライズされるケースがあり、単回参照の中間表は**派生表**に畳んだ方が速いことが多いです（上の「3）」がそれ）。

3. **LeetCode の Runtime はノイズ大**
   実環境では **EXPLAIN** で実行計画を確認し、`rows` 見積もり・ファイルソート有無・テンポラリ使用などをチェックしてく さい。

---

## まとめ

* **一意最大前提**なら ⇒ `GROUP BY ... ORDER BY COUNT(*) DESC LIMIT 1` が最有力。
* **同率最大も返す**なら ⇒ `HAVING COUNT(*) = (SELECT ... LIMIT 1)` がシンプル＆速いことが多い。
* **ウィンドウ採用**なら ⇒ 中間 CTE 省略の 1 段構成に。
* **物理対策** ⇒ `customer_number` にインデックス。
